# Part5

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import requests, json, os, sys, time, re
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import pandas as pd

In [54]:
######## Spark Session是关系型功能的入口点  #############
spark = SparkSession \
    .builder \
    .appName('spark-rec') \
    .config("configuration_key", "configuration_value") \
    .enableHiveSupport() \
    .getOrCreate()

In [55]:
sc = spark.sparkContext

In [73]:
import os
import math
import datetime
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType
datasets_path=os.getcwd() + "/RE_data"
#ratings_file = os.path.join(datasets_path, 'data/ratings.csv')
ratings_raw_data = sc.textFile("df_col_use_app_raings_new.csv")
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_data = ratings_raw_data.filter(lambda line: line != ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[1]),int(tokens[2]),int(float(tokens[3])))).cache()

rddTraining, rddValidating, rddTesting = ratings_data.randomSplit([6,2,2], seed=1001)

* Build the Spark ALS model

In [74]:
model = ALS.trainImplicit(rddTraining, 10, 10,alpha=0.01)
testset = sc.parallelize([(3, 4000), (3, 15700)])
model.predictAll(testset).collect()

[Rating(user=3, product=15700, rating=0.987999240898132),
 Rating(user=3, product=4000, rating=0.9856218982089308)]

In [82]:
# Calculate all predictions
rddTesting_map = rddTesting.map(lambda r: ((r[0], r[1]))) 
predictions = model.predictAll(rddTesting_map).map(lambda r: ((r[0], r[1]), (r[2]))) 
predictions.take(5)    ####### Output 5 results
#model.predictAll(rddTesting_map).collect()


[((6, 304050), 0.0013580463704940815),
 ((0, 377160), 0.006895625075594436),
 ((0, 863550), 0.00023213074238198628),
 ((3, 863550), 0.00408147803901715),
 ((2, 239140), 0.007940296344380648)]

In [76]:
rates_and_preds = rddTesting.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions) 
rates_and_preds.take(4)



[((0, 30), (3.0, 0.0067794034783182355)),
 ((0, 550), (5.0, 0.003280047368061534)),
 ((0, 1046930), (2.0, 0.004346375003993819)),
 ((0, 304930), (1.0, 0.006519364686702334))]

* Show the final top 10 recommendation for 8 users 0 - 7

In [93]:
Spark_rec_list = []
for i in range(8):
    Spark_rec_list.append(model.recommendProducts(i,10))
Spark_rec_df = pd.DataFrame(Spark_rec_list)

Spark_rec_df

,0,1,2,3,4,5,6,7,8,9
0,"(0, 10, 1.001914868207123)","(0, 230410, 1.001914868207123)","(0, 261550, 1.0018208213080169)","(0, 582010, 1.0018208213080169)","(0, 380, 0.9941490030218183)","(0, 238960, 0.9937153434673996)","(0, 500, 0.9919288380739991)","(0, 271590, 0.9917913378137204)","(0, 363970, 0.9897275833099256)","(0, 239140, 0.9865745227133966)"
1,"(1, 236390, 1.0039952483163501)","(1, 204360, 1.0038837804763279)","(1, 255710, 1.0037701741024045)","(1, 304050, 1.0035362421205676)","(1, 1046930, 0.9980763426994006)","(1, 218620, 0.9934348552581811)","(1, 359550, 0.9927907100365007)","(1, 578080, 0.9867839083026121)","(1, 8930, 0.9848420152806818)","(1, 322330, 0.9840510539531147)"
2,"(2, 224260, 1.0001989628397978)","(2, 292030, 0.9982868971287413)","(2, 72850, 0.9927436029731327)","(2, 1085660, 0.9910489035002124)","(2, 620, 0.9897018844830188)","(2, 271590, 0.9896831620183868)","(2, 730, 0.9895648226827858)","(2, 377160, 0.989134985039848)","(2, 431960, 0.9888925331922738)","(2, 105600, 0.9887285713189122)"
3,"(3, 219740, 1.0032542154077184)","(3, 278360, 1.0029622962026936)","(3, 130, 1.0029622962026936)","(3, 291480, 1.0028611926873627)","(3, 96000, 1.0001689105194427)","(3, 242760, 0.9964551625149138)","(3, 550, 0.9935908720892108)","(3, 304930, 0.9917885595356373)","(3, 620, 0.9904832352684103)","(3, 15700, 0.987999240898132)"
4,"(4, 240, 0.9993494778492729)","(4, 555570, 0.9993494778492729)","(4, 381210, 0.9993494778492729)","(4, 227300, 0.9993494778492729)","(4, 320, 0.9993494778492729)","(4, 370910, 0.9993494778492729)","(4, 242760, 0.999248621586452)","(4, 863550, 0.9992376021220086)","(4, 221380, 0.999123600540412)","(4, 420, 0.999123600540412)"
5,"(5, 220, 1.0012194827135477)","(5, 96000, 0.9995773624525937)","(5, 377160, 0.9991087389423341)","(5, 15700, 0.9938236661461112)","(5, 252950, 0.9930493164505234)","(5, 10180, 0.9930480258929044)","(5, 208090, 0.9922480186519411)","(5, 550, 0.9910267632749773)","(5, 219640, 0.9907244431093817)","(5, 304930, 0.9895819917615831)"
6,"(6, 291550, 0.9996421226467529)","(6, 813820, 0.9995527415800678)","(6, 700330, 0.9995527415800678)","(6, 444090, 0.9993687208224468)","(6, 438100, 0.9992740024089826)","(6, 252950, 0.9941588918956608)","(6, 10180, 0.9935627308622328)","(6, 218620, 0.992101641849176)","(6, 359550, 0.9917595607253391)","(6, 1085660, 0.9913506573525135)"
7,"(7, 413150, 1.002100938316386)","(7, 1046930, 0.9981470932439502)","(7, 44350, 0.995827961139858)","(7, 30, 0.9958269964485482)","(7, 363970, 0.9919097660168095)","(7, 500, 0.9915687787906834)","(7, 377160, 0.9912958931263443)","(7, 219640, 0.984215290252194)","(7, 8930, 0.9841061906628886)","(7, 252490, 0.9832909875458935)"


* Mesuare our recommendations by applying RMSE

In [79]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) 
print ('For testing data the RMSE is %s' % (error))


For testing data the RMSE is 3.605158405851843
